In [ ]:
import os
import re
from datetime import datetime
from os import path
from typing import List, Dict

from commonroad.common.util import Interval, AngleInterval

from BatchConversion.BatchConverter import BatchConverter, BatchConversionResult
from OpenSCENARIO2CR.AbsRel import AbsRel
from OpenSCENARIO2CR.ConversionStatistics import EAnalyzer
from OpenSCENARIO2CR.EsminiWrapper.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason

In [ ]:
from os.path import expanduser

batch_converter = BatchConverter(
    converter_class=Osc2CrConverter,

    delta_t=0.1,
    goal_state_position_width=5,
    goal_state_position_length=25,
    goal_state_position_use_ego_rotation=True,
    goal_state_time_step=AbsRel[Interval](Interval(-5, 0), is_absolute=False),
    goal_state_orientation=AbsRel[AngleInterval](AngleInterval(-0.8, 0.8), is_absolute=False),

    keep_ego_vehicle=True,
    do_run_cr_monitor=True,
    do_trim_scenario=False,

    ego_filter=re.compile(r".*ego.*", re.IGNORECASE),

    use_implicit_odr_file=True,

    max_time=120.0,
    grace_time=1.0,
    ignored_level=EStoryBoardElementType.ACT,
    log_to_console=False,
    log_to_file=False,

    analyzers=[EAnalyzer.STL_MONITOR, EAnalyzer.DRIVE_ABILITY_CHECKER]
)

batch_converter.discover_files(expanduser("~"), re.compile(r".*\.xosc", re.IGNORECASE), recursively=True)

In [ ]:
time_conversion_started = datetime.now()
results: Dict[str, BatchConversionResult] = batch_converter.run_batch_conversion(num_worker=None)
time_conversion_finished = datetime.now()

In [ ]:
from commonroad.scenario.scenario import Tag
from commonroad.common.file_writer import CommonRoadFileWriter, OverwriteExistingFile
import yaml

storage_path = "results/{}".format(time_conversion_finished.isoformat(sep="_", timespec="seconds"))
store = True

if store:
    if not path.exists(storage_path):
        os.mkdir(storage_path)
    statistics_dict = {}
    written_scenarios: List[str] = []
    for scenario_path, result in results.items():
        basename = path.basename(scenario_path)
        path_no_extension = path.join(storage_path, path.splitext(basename)[0])
        if path_no_extension in written_scenarios:
            i = 1
            while True:
                if (new_path := path_no_extension + f"-{i}") not in written_scenarios:
                    path_no_extension = new_path
                    break
                else:
                    i += 1
        written_scenarios.append(path_no_extension)

        if result.without_exception:
            if isinstance(result.conversion_result, EFailureReason):
                statistics_dict[path_no_extension] = ("failure", result.conversion_result.name)
            else:
                scenario, planning_problem_set, statistics = result.conversion_result
                statistics_dict[path_no_extension] = ("success", statistics.to_dict())
                CommonRoadFileWriter(
                    scenario=scenario,
                    planning_problem_set=planning_problem_set,
                    author="ADD AUTHOR HERE",
                    affiliation="ADD AFFILIATION HERE",
                    source="ADD SOURCE HERE",
                    tags={Tag.SIMULATED},
                ).write_to_file(path_no_extension + ".xml", OverwriteExistingFile.ALWAYS)
        else:
            statistics_dict[path_no_extension] = ("error", result.exception_text, result.traceback)

    with open(path.join(storage_path, "statistics.yml"), "w") as file:
        yaml.safe_dump(statistics_dict, file)


In [ ]:

from OpenSCENARIO2CR.ConversionAnalyzer.ErrorAnalysisResult import ErrorAnalysisResult

errors: Dict[ErrorAnalysisResult, int] = {}
for result in results.values():
    if result.without_exception:
        if not isinstance(result.conversion_result, EFailureReason):
            scenario, planning_problem_set, statistics = result.conversion_result
            for analysis_type, analysis_result in statistics.analysis.items():
                for vehicle_name, vehicle_result in analysis_result.items():
                    if isinstance(vehicle_result, ErrorAnalysisResult):
                        if vehicle_result in errors:
                            errors[vehicle_result] += 1
                        else:
                            errors[vehicle_result] = 1

for error, count in errors.items():
    print(f"{count}\n{error.traceback_text}")
    print("\n" * 3)